In [ ]:
!pip install oxenai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: polars
    Found existing installation: polars 1.31.0
    Uninstalling polars-1.31.0:
      Successfully uninstalled polars-1.31.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uni

In [ ]:
import os
import pandas as pd
from oxen import RemoteRepo
import re

In [ ]:
from google.colab import userdata
oxen_key = userdata.get('oxen_key')

from oxen.auth import config_auth
config_auth(oxen_key)

In [ ]:
def download_data(repo_name: str, file_name: str):
    print(f"Downloading: {file_name}")
    repo = RemoteRepo(repo_name)

    if os.path.exists(file_name):
        return True

    try:
        repo.download(file_name)
        return True
    except Exception as e:
        return False

# Caculate Accuracy

In [ ]:
def extract_column(df, src_col, extract_col):
    df[extract_col] = df[src_col].str.extract(f'<{extract_col}>(.*?)</{extract_col}>', flags=re.DOTALL)[0]
    df[extract_col] = df[extract_col].str.strip()
    return df

In [ ]:
def extract_data(file_name: str):
    if file_name.endswith("parquet"):
        df = pd.read_parquet(file_name)
    if file_name.endswith("jsonl"):
        df = pd.read_json(file_name, lines=True)
    df = extract_column(df, 'judgement', 'answer')
    return df

In [ ]:
def calculate_accuracy(df):
    agg = df['answer'].value_counts()
    true_count = agg.get('true', 0)
    total_count = len(df['answer'])
    accuracy = true_count / total_count if total_count > 0 else 0.0
    return true_count, total_count, accuracy

# Download parquet file from oxen repo and Calculate accuracy for list models

In [ ]:
# Repo name in Oxen
repo_name = "ZiDuck/text2sql"

# List of models and equivalent judgement files
model_files = [
    {'name': 'GPT-4o Single-Table', 'file_path': 'results/GPT-4o-Single-Table-Judgement.parquet'},
    {'name': 'Original-Qwen3-0.6B Single-Table', 'file_path': 'results/Qwen3-0.6B-Single-Table-Predictions.parquet'},
    {'name': 'SFT-Qwen3-0.6B Single-Table', 'file_path': 'results/SFT-Qwen3-0.6B-Text2SQL-SingleTable'},
    {'name': 'GPT-4o BIRD', 'file_path': 'results/GPT-4o-Judgements_with_question.parquet'},
    {'name': 'Original-Qwen3-0.6B BIRD', 'file_path': 'results/Qwen3-0.6B-Judgements.parquet'},
    {'name': 'SFT-Qwen3-0.6B BIRD', 'file_path': 'results/SFT-Qwen3-0.6B-Judgements.parquet'},
]


In [ ]:
results = []

for model in model_files:
    name = model['name']
    file_path = model['file_path']

    print(f"\n=== Evaluating model: {name} ===")

    ok = download_data(repo_name, file_path)
    if not ok:
        print(f"  → Skipped {name} (file not found or download failed).")
        continue

    try:
        df = extract_data(file_path)
        if df is None or len(df) == 0:
            print(f" ERROR: File is empty or unreadable → Skipped.")
            continue
    except Exception as e:
        print(f" ERROR reading {file_path}: {e}")
        continue

    try:
        true_count, total_count, acc = calculate_accuracy(df)
        print(f"{name}: {true_count}/{total_count} = {acc*100:.2f}%")
    except Exception as e:
        print(f"ERROR calculating accuracy: {e}")
        continue

    results.append({
        'model': name,
        'true': true_count,
        'total': total_count,
        'accuracy': acc
    })



=== Evaluating model: GPT-4o Single-Table ===
Downloading: results/GPT-4o-Single-Table-Judgement.parquet
GPT-4o Single-Table: 90/200 = 45.00%

=== Evaluating model: Original-Qwen3-0.6B Single-Table ===
Downloading: results/Qwen3-0.6B-Single-Table-Predictions.parquet
Original-Qwen3-0.6B Single-Table: 28/200 = 14.00%

=== Evaluating model: SFT-Qwen3-0.6B Single-Table ===
Downloading: results/Qwen3-0.6B.parquet
SFT-Qwen3-0.6B Single-Table: 98/200 = 49.00%

=== Evaluating model: GPT-4o BIRD ===
Downloading: results/GPT-4o-Judgements_with_question.parquet
GPT-4o BIRD: 311/500 = 62.20%

=== Evaluating model: Original-Qwen3-0.6B BIRD ===
Downloading: results/Qwen3-0.6B-Judgements.parquet
Original-Qwen3-0.6B BIRD: 33/500 = 6.60%

=== Evaluating model: SFT-Qwen3-0.6B BIRD ===
Downloading: results/SFT-Qwen3-0.6B-Judgements.parquet
SFT-Qwen3-0.6B BIRD: 10/500 = 2.00%


In [ ]:
pd.DataFrame(results)

,model,true,total,accuracy
0,GPT-4o Single-Table,90,200,0.450
1,Original-Qwen3-0.6B Single-Table,28,200,0.140
2,SFT-Qwen3-0.6B Single-Table,98,200,0.490
3,GPT-4o BIRD,311,500,0.622
4,Original-Qwen3-0.6B BIRD,33,500,0.066
5,SFT-Qwen3-0.6B BIRD,10,500,0.020
